In [12]:
import pandas as pd
from sklearn.pipeline import Pipeline

In [13]:
# Load data for badminton
data_badminton = pd.read_csv(r"C:\Users\Pavan Kalyan\Desktop\DS_intership\Task 6\reviews_badminton\data.csv")

# Load data for tea
data_tea = pd.read_csv(r"C:\Users\Pavan Kalyan\Desktop\DS_intership\Task 6\reviews_tea\data.csv")
# Load data for tawa
data_tawa = pd.read_csv(r"C:\Users\Pavan Kalyan\Desktop\DS_intership\Task 6\reviews_tawa\data.csv")

In [14]:
# EDA for data_badminton
print("EDA for data_badminton:")
print(data_badminton.info())
print(data_badminton.describe())

# EDA for data_tawa
print("\nEDA for data_tawa:")
print(data_tawa.info())
print(data_tawa.describe())

# EDA for data_tea
print("\nEDA for data_tea:")
print(data_tea.info())
print(data_tea.describe())

EDA for data_badminton:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8518 entries, 0 to 8517
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Reviewer Name    8508 non-null   object 
 1   Review Title     8508 non-null   object 
 2   Place of Review  8468 non-null   object 
 3   Up Votes         8508 non-null   float64
 4   Down Votes       8508 non-null   float64
 5   Month            8053 non-null   object 
 6   Review text      8510 non-null   object 
 7   Ratings          8518 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 532.5+ KB
None
          Up Votes   Down Votes      Ratings
count  8508.000000  8508.000000  8518.000000
mean      0.391396     0.121768     4.181028
std      11.613909     3.248022     1.262200
min       0.000000     0.000000     1.000000
25%       0.000000     0.000000     4.000000
50%       0.000000     0.000000     5.000000
75%       0.000000     0.0

In [15]:
# Rename columns to have consistent names
data_badminton.rename(columns={'Reviewer Name': 'reviewer_name',
                                'Review Title': 'review_title',
                                'Place of Review': 'place_of_review',
                                'Up Votes': 'up_votes',
                                'Down Votes': 'down_votes',
                                'Month': 'date_of_review',
                                'Review text': 'review_text',
                                'Ratings': 'reviewer_rating'}, inplace=True)

data_tawa.rename(columns={'Reviewer_Name': 'reviewer_name',
                            'Reviewer_Rating': 'reviewer_rating',
                            'Review_Title': 'review_title',
                            'Review_Text': 'review_text',
                            'Place_of_Review': 'place_of_review',
                            'Date_of_Review': 'date_of_review',
                            'Up_Votes': 'up_votes',
                            'Down_Votes': 'down_votes'}, inplace=True)

data_tea.rename(columns={'reviewer_rating': 'reviewer_rating',
                            'reviewer_name': 'reviewer_name',
                            'review_title': 'review_title',
                            'review_text': 'review_text',
                            'place_of_review': 'place_of_review',
                            'Date_of_review': 'date_of_review',
                            'up_votes': 'up_votes',
                            'Down_votes': 'down_votes'}, inplace=True)

In [16]:
# Add a 'Product' column to each dataset
data_badminton['Product'] = 'Badminton'
data_tawa['Product'] = 'Tawa'
data_tea['Product'] = 'Tea'

# Concatenate datasets for easier analysis
data_combined = pd.concat([data_badminton, data_tawa, data_tea], ignore_index=True)

In [17]:
data_combined.columns

Index(['reviewer_name', 'review_title', 'place_of_review', 'up_votes',
       'down_votes', 'date_of_review', 'review_text', 'reviewer_rating',
       'Product'],
      dtype='object')

In [18]:
data_combined.shape

(20219, 9)

In [19]:
data_combined['review_text'] = data_combined['review_text'].fillna('')
data_combined['review_text'].isnull().sum()

0

In [55]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')



def lowers_text(x):
    return x.str.lower()
def special_char(x):
    return x.apply(lambda x : re.sub(r'[^\w\s]',"",x))
def Digits(x):
    return x.apply(lambda x: re.sub(r'\d+', '',x))
def lemma(x):
    from nltk.tokenize import word_tokenize as wt
    list_stp = stopwords.words('english')
    wl = WordNetLemmatizer()
    
    def lemmatize_text(text):
        words = wt(text)
        lemmatized_words = [wl.lemmatize(word,pos="v") for word in words if word not in list_stp ]
        return " ".join(lemmatized_words)
    return x.apply(lemmatize_text)


In [58]:
from sklearn.preprocessing import FunctionTransformer
        

# Creating Pipeline for Text_cleaning
Pre_process_pip = Pipeline([
    ('Lower', FunctionTransformer(lowers_text, validate=False)),
    ('Digits', FunctionTransformer(Digits, validate=False)),
    ('Special_characters', FunctionTransformer(special_char, validate=False)),
    ('Advance', FunctionTransformer(lemma, validate=False))
])



In [59]:

# Apply preprocessing function to the 'review_text' column

data_combined['cleaned_review_text']=Pre_process_pip.transform(data_combined['review_text'])

# Preview the preprocessed text
print(data_combined[['review_text', 'cleaned_review_text']].head())

                                         review_text  \
0  Nice product, good quality, but price is now r...   
1  They didn't supplied Yonex Mavis 350. Outside ...   
2  Worst product. Damaged shuttlecocks packed in ...   
3  Quite O. K. , but nowadays  the quality of the...   
4  Over pricedJust â?¹620 ..from retailer.I didn'...   

                                 cleaned_review_text  
0  nice product good quality price rise bad sign ...  
1  didnt supply yonex mavis outside cover yonex a...  
2  worst product damage shuttlecock pack new box ...  
3  quite k nowadays quality cork like years back ...  
4  pricedjust â¹ retaileri didnt understand wat a...  


In [61]:
# Serialisation of Text_preprocess pipeline
import pickle
with open(r"Application_Deployment\models\Text_pip.pkl", 'wb') as file:
    pickle.dump(Pre_process_pip, file)

In [34]:
data_combined.columns

Index(['reviewer_name', 'review_title', 'place_of_review', 'up_votes',
       'down_votes', 'date_of_review', 'review_text', 'reviewer_rating',
       'Product', 'cleaned_review_text'],
      dtype='object')

In [35]:
data_combined.head()

,reviewer_name,review_title,place_of_review,up_votes,down_votes,date_of_review,review_text,reviewer_rating,Product,cleaned_review_text
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4.0,Badminton,nice product good quality price rise bad sign ...
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1.0,Badminton,didnt supply yonex mavis outside cover yonex a...
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1.0,Badminton,worst product damage shuttlecock pack new box ...
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3.0,Badminton,quite k nowadays quality cork like years back ...
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1.0,Badminton,pricedjust â¹ retaileri didnt understand wat a...


In [36]:
"".join(data_combined['cleaned_review_text'][0])

'nice product good quality price rise bad sign affordable price especially play everyday kindly help us term price thank youread'

In [37]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Intializing SentimentIntensityAnalyzer object
analyzer = SentimentIntensityAnalyzer()

# Define a function to analyze the sentiment of each review text
def analyze_sentiment(cleaned_text):
    compound_score = analyzer.polarity_scores(cleaned_text)['compound']
    
    # Assign positive sentiment if compound score is greater than 0, else negative sentiment
    if compound_score >0:
        return 1 # Positive
    else:
        return 0 # Negative
    
# Apply the function to the 'cleaned_review_text' column
data_combined['sentiments'] = data_combined['cleaned_review_text'].apply(analyze_sentiment)

# Display the DataFrame with the new 'sentiments' column
data_combined.head()

,reviewer_name,review_title,place_of_review,up_votes,down_votes,date_of_review,review_text,reviewer_rating,Product,cleaned_review_text,sentiments
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4.0,Badminton,nice product good quality price rise bad sign ...,1
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1.0,Badminton,didnt supply yonex mavis outside cover yonex a...,0
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1.0,Badminton,worst product damage shuttlecock pack new box ...,0
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3.0,Badminton,quite k nowadays quality cork like years back ...,1
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1.0,Badminton,pricedjust â¹ retaileri didnt understand wat a...,0


In [38]:
data_combined['sentiments'].value_counts(dropna =False,normalize=True)

1    0.634749
0    0.365251
Name: sentiments, dtype: float64

## Step 1 - Identify Input and Output

In [39]:
X = data_combined['review_text']
y = data_combined['sentiments']
print(X.shape, y.shape)

(20219,) (20219,)


## Step 2 - Split the Data into Train and Test

In [40]:
# Step 2: Split the data into train and test

from sklearn.model_selection import train_test_split

# split X and y into training and testing sets. 
# By default, it splits 75% training and 25% test
# random_state=1 for reproducibility

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(15164,) (15164,)
(5055,) (5055,)


### Caching to optimize the runtime
We can create the Memory object to cache the intermediate results of `vectorization`.

This will avoid redundant computations of vectorization during the grid search, thereby optimizing the runtime. Adjust the cachedir parameter as needed to specify the location for caching the results.

### Implementing Memoization to further improve the runtime
Implement memoization to store the results of `expensive function calls` and reuse them when the same inputs occur again. This is particularly useful for dynamic programming problems.

## Step 4 - Text cleaning of Train data


In [41]:
# Improving the efficiency by applying cleaning the text data before hand

%time X_train_clean = Pre_process_pip.transform(X_train)

CPU times: total: 1.09 s
Wall time: 2.55 s


## Step 5 - Text cleaning of Test data

In [42]:
%time X_test_clean = Pre_process_pip.transform(X_test)

CPU times: total: 500 ms
Wall time: 867 ms


In [43]:

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
import pickle

# Define a list of classifiers to try
classifiers = [
    ('Logistic Regression', LogisticRegression()),
    ('Random Forest', RandomForestClassifier()),
    ('Naive Bayes', MultinomialNB())
]

best_model = None
best_f1_score = 0

# Iterate over classifiers
for clf_name, clf in classifiers:
    # Define the pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', clf)
    ])
    
    # Train the pipeline
    pipeline.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = pipeline.predict(X_test)
    
    # Calculate F1-score
    f1 = f1_score(y_test, y_pred)
    print(f"F1-score for {clf_name}: {f1}")
    
    # Check if this model is the best so far
    if f1 > best_f1_score:
        best_f1_score = f1
        best_model = pipeline

# Pickle the best model
with open(r'Application_Deployment\models\best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)


F1-score for Logistic Regression: 0.974023953958625
F1-score for Random Forest: 0.9719684063806722
F1-score for Naive Bayes: 0.9482599322451494


In [47]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# Define a list of classifiers to evaluate
classifiers = [
    ('Logistic Regression', LogisticRegression()),
    ('Random Forest', RandomForestClassifier()),
    ('Naive Bayes', MultinomialNB()),
     ('Deciion Tree',DecisionTreeClassifier())
     ]

best_f1_score = 0
best_model = None

# Iterate over different classifiers
for name, classifier in classifiers:
    # Define the pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),  # TF-IDF Vectorization
        ('clf', classifier)  # Classifier
    ])
    
    # Train the model
    
    %time pipeline.fit(X_train, y_train)
    
    # Predictions on training and testing sets
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # Calculate F1-score on training and testing sets
    train_f1_score = f1_score(y_train, y_train_pred)
    test_f1_score = f1_score(y_test, y_test_pred)
    
    print(f"Classifier: {name}")
    print(f"Train F1-score: {train_f1_score:.4f}")
    print(f"Test F1-score: {test_f1_score:.4f}")
    print()
    
    # Check if the current model has the best F1-score on the testing set
    if test_f1_score > best_f1_score:
        best_f1_score = test_f1_score
        best_model = pipeline

# Pickle the best model
import pickle

pickle.dump(best_model,open(r"Application_Deployment\models\best_model.pkl",'wb'))

CPU times: total: 78.1 ms
Wall time: 269 ms
Classifier: Logistic Regression
Train F1-score: 0.9826
Test F1-score: 0.9740

CPU times: total: 7.19 s
Wall time: 10.5 s
Classifier: Random Forest
Train F1-score: 0.9966
Test F1-score: 0.9707

CPU times: total: 125 ms
Wall time: 172 ms
Classifier: Naive Bayes
Train F1-score: 0.9669
Test F1-score: 0.9483

CPU times: total: 547 ms
Wall time: 695 ms
Classifier: Deciion Tree
Train F1-score: 0.9966
Test F1-score: 0.9698



In [48]:
import pickle

# Load the pickled model
with open('best_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Take user input text
user_input_text = input("Enter your text: ")
Review = pd.Series(user_input_text)

# Preprocess the input text
processed_input_text = Pre_process_pip.transform(Review)  

# Use the loaded model to predict the sentiment of the input text
predicted_sentiment = model.predict(processed_input_text)

# Map predicted sentiment to human-readable labels
predicted_sentiment_label = "Positive" if predicted_sentiment[0] == 1 else "Negative"

# Print the predicted sentiment
print("Predicted sentiment:", predicted_sentiment_label)


Enter your text: good product
Predicted sentiment: Positive
